#**Step 1: installazione delle librerie**

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
ERROR: pip's dependency r

#**Step 2: caricamento delle librerie**

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

import pandas as pd
from datasets import Dataset
import re
import random

#**Step 3: impostazione delle variabili**

In [20]:
percentuale_dataset = 1

# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-7b-hf"

# Number of training epochs
num_train_epochs = 2

# Fine-tuned model name
#rand = random.randint(0, 10000)
#rand = 1824
new_model = f"meta-llama2_7b_ge_{num_train_epochs}epoch"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

#**Step 4: caricamento del dataset**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

# change this to match your path
data_path_models = "./drive/MyDrive/Colab Notebooks/WebNLG-IT/Esperimento 1b/Models"
data_path_data = "./drive/MyDrive/Colab Notebooks/WebNLG-IT/Esperimento 1b/Datasets"

Mounted at /content/drive


In [21]:
# carico il dataset
trainset = pd.read_csv(f'{data_path_data}/WebNLG-GE/train.csv')
devset = pd.read_csv(f'{data_path_data}/WebNLG-GE/dev.csv')

# ottengo un sottoinsieme del dataset di partenza
subset_trainset = trainset.sample(frac=percentuale_dataset, random_state=42)
subset_trainset.head()

,data_unit,sentence
15981,Bill_Oddie birthPlace Rochdale Bananaman broad...,Rochdale geboren Bill Oddie spielte in Bananam...
7814,Addis_Ababa_City_Hall country Ethiopia Addis_A...,"Das Stadion von Addis Abeba, benannt nach dem ..."
150,"Alan_Shepard almaMater ""NWC, M.A. 1957"" Alan_S...",New Hampshire born Alan Shepard (Nov 18 1923) ...
1684,Allama_Iqbal_International_Airport cityServed ...,Die pakistanische Zivilluftfahrtbehörde befind...
7497,Bhajji country India Bhajji ingredient Gram_fl...,Die wichtigsten Zutaten eines Bhajji aus Indie...


In [22]:
len(subset_trainset)

15992

In [23]:
# ottengo un sottoinsieme del dataset di partenza
subset_devset = devset.sample(frac=percentuale_dataset, random_state=42)
subset_devset.head()

,data_unit,sentence
65,"Bacon_sandwich alternativeName ""Bacon butty, b...",Speck-Brötchen sind in Großbritannien beliebt ...
940,Blockbuster_(comicsCharacter) alternativeName ...,Tom Lyle und Roger Stern schufen die Comicfigu...
1722,California language Vietnamese_language,"Vietnamesisch ist eine Sprache, die in Kalifor..."
1698,Malaysia capital Kuala_Lumpur Asam_pedas count...,Malaysia und die malaiische Halbinsel beherber...
799,British_Hong_Kong representative Chris_Patten,Chris Patten vertrat das britische Hongkong.


In [24]:
llama_trainset = []
for index, row in subset_trainset.iterrows():
    triple = row['data_unit']
    sentence = row['sentence']

    llama_trainset.append(f'<s> [INST] Given the following triples in (TRIPLE), you have to generate the corresponding text in (ANW) [/INST] [TRIPLE] {triple} [/TRIPLE] [ANW] {sentence} [/ANW] </s>')

llama_trainset = pd.DataFrame(llama_trainset, columns=['instr'])
llama_trainset.head()

,instr
0,<s> [INST] Given the following triples in (TRI...
1,<s> [INST] Given the following triples in (TRI...
2,<s> [INST] Given the following triples in (TRI...
3,<s> [INST] Given the following triples in (TRI...
4,<s> [INST] Given the following triples in (TRI...


In [25]:
llama_trainset = Dataset.from_pandas(llama_trainset)

In [26]:
llama_devset = []
for index, row in subset_devset.iterrows():
    triple = row['data_unit']
    sentence = row['sentence']

    llama_devset.append(f'<s> [INST] Given the following triples in (TRIPLE), you have to generate the corresponding text in (ANW) [/INST] [TRIPLE] {triple} [/TRIPLE] [ANW] {sentence} [/ANW] </s>')

llama_devset = pd.DataFrame(llama_devset, columns=['instr'])
llama_devset.head()

,instr
0,<s> [INST] Given the following triples in (TRI...
1,<s> [INST] Given the following triples in (TRI...
2,<s> [INST] Given the following triples in (TRI...
3,<s> [INST] Given the following triples in (TRI...
4,<s> [INST] Given the following triples in (TRI...


In [27]:
llama_devset = Dataset.from_pandas(llama_devset)

#**Step 5: caricamento del modello**

In [12]:
!pip install huggingface_hub

In [13]:
from huggingface_hub import login
token = ""
login(token=token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [28]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

Your GPU supports bfloat16: accelerate training with bf16=True


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

#**Step 6: fine-tuning del modello**

In [ ]:
del training_arguments

In [29]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    evaluation_strategy="steps",  # Calcolo della validation loss ad ogni step
    save_strategy="steps",  # Checkpoint del modello ad ogni step
    save_total_limit=3,  # Limita il numero di checkpoint salvati
    load_best_model_at_end=True,  # Carica il modello migliore al termine dell'addestramento
    metric_for_best_model="eval_loss",  # Sceglie la metrica per determinare il miglior modello
    greater_is_better=False,  # Indica se un valore più alto della metrica è migliore o no
    eval_steps=500,  # Numero di passaggi prima di valutare il modello
    logging_steps= 500,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=llama_trainset,
    eval_dataset=llama_devset,
    peft_config=peft_config,
    dataset_text_field="instr",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/15992 [00:00<?, ? examples/s]

Map:   0%|          | 0/1776 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
500,0.821600,0.492008
1000,0.411700,0.388214
1500,0.347000,0.341726
2000,0.314500,0.311150
2500,0.296200,0.296169
3000,0.284200,0.279499


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning

Step,Training Loss,Validation Loss
500,0.821600,0.492008
1000,0.411700,0.388214
1500,0.347000,0.341726
2000,0.314500,0.311150
2500,0.296200,0.296169
3000,0.284200,0.279499
3500,0.271600,0.270473
4000,0.261600,0.263580
4500,0.237800,0.256667
5000,0.236500,0.249735


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning

TrainOutput(global_step=7996, training_loss=0.30164954589568954, metrics={'train_runtime': 14572.5726, 'train_samples_per_second': 2.195, 'train_steps_per_second': 0.549, 'total_flos': 9.608023324950528e+16, 'train_loss': 0.30164954589568954, 'epoch': 2.0})

In [30]:
# Sostituisci "path_to_save" con il percorso desiderato nel tuo drive
trainer.save_model(f'{data_path_models}/{new_model}')

#**Step 7: generazione frasi del modello creato**

In [34]:
# Empty VRAM
import gc
gc.collect()
gc.collect()

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, f'{data_path_models}/{new_model}')
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [35]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

def generate_text(triple, actual):
  max_length = len(f"<s>[INST] Given the following triples in (TRIPLE), you have to generate the corresponding text in (ANW) [/INST] [TRIPLE] {triple} [/TRIPLE] [ANW] {actual} [/ANW]") * 1.5

  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=max_length, temperature=0.1)
  result = pipe(f"<s>[INST] Given the following triples in (TRIPLE), you have to generate the corresponding text in (ANW) [/INST] [TRIPLE] {triple} [/TRIPLE]")

  if result is not None and result and len(result) > 0 and 'generated_text' in result[0]:
      generated_text = result[0]['generated_text']

      # Utilizziamo una regex per estrarre il contenuto all'interno del primo tag [ANW] ... [/ANW]
      match = re.search(r'\[ANW\] (.*?) \[/ANW\]', generated_text)

      if match:
          content_within_anw_tag = match.group(1)
          text = content_within_anw_tag
      else:
          text = generated_text

  else:
      text = "Il risultato della pipeline è nullo."

  return text

In [36]:
percentuale_dataset = 1

In [37]:
testset = pd.read_csv(f'{data_path_data}/WebNLG-GE/test.csv')

# ottengo un sottoinsieme del testset di partenza
subset_testset = testset.sample(frac=percentuale_dataset, random_state=42)
print(len(subset_testset))

# test
X_test = subset_testset['data_unit'].values.tolist()
y_test = subset_testset['sentence'].values.tolist()

1000


In [38]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="torch.utils.checkpoint")
warnings.filterwarnings("ignore", category=UserWarning, message="None of the inputs have requires_grad=True")

In [39]:
triples = []
predictions = []
actuals = []

i = 0
for tripla in X_test:
  if i < 1010:
    actual = y_test[i]
    prediction = generate_text(tripla, actual)

    triples.append(tripla)
    actuals.append(actual)
    predictions.append(prediction)

    print(f'======== STEP {i} ======== ')
    print(f'Tripla: {tripla}')
    print(f'Actual: {actual}')
    print(f'Prediction: {prediction}')
    print('\n')

    df = pd.DataFrame(list(zip(triples, predictions, actuals)), columns =['triples', 'predictions', 'actuals'])
    df.to_csv(f'{data_path_models}/{new_model}_decoding.csv', index=False)

  i = i + 1

Output streaming troncato alle ultime 5000 righe.

======== STEP 167 ======== 
Tripla: Andrews_County_Airport runwayLength 1773.0
Actual: Die Länge der Start- und Landebahn am Flughafen Andrews County beträgt 1773 Meter. 
Prediction: Die Start- und Landebahn von Andrews County Airport ist 1773,0. 


======== STEP 168 ======== 
Tripla: Alderney_Airport runwayLength 497.0
Actual: Die Fahrbahnlänge des Flughafens Alderney beträgt 497,0. 
Prediction: Die Fahrbahnlänge des Alderney Airport beträgt 497,0. 


======== STEP 169 ======== 
Tripla: A.C._Chievo_Verona ground "Verona, Italy" A.C._Chievo_Verona league Serie_A
Actual: Verona, Italien ist die Heimat von A.C. Chievo Verona, die in der Liga, Serie A. 
Prediction: <s>[INST] Given the following triples in (TRIPLE), you have to generate the corresponding text in (ANW) [/INST] [TRIPLE] A.C._Chievo_Verona ground "Verona, Italy" A.C._Chievo_Verona league Serie_A [/TRIPLE] [ANW] A.C. Chievo Verona spielen in der Serie A und ihr Boden ist in Ve